In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm.notebook import tqdm
import json
import pandas as pd

In [3]:
def get_listing_details(url_row):

    r = requests.get(url_row[1])
    soup = BeautifulSoup(r.content)
    
    if soup.find_all('p', class_="wt-text-body-01 wt-mr-xs-1"):
        if soup.find('p', class_="wt-text-body-01 wt-mr-xs-1").find_all('a', class_="wt-text-link-no-underline"):
            seller_text = soup.find('p', class_="wt-text-body-01 wt-mr-xs-1").find('a', class_="wt-text-link-no-underline").text.strip()
        else:
            seller_text = None
    else:
        seller_text = None
        
    if soup.find_all('p', class_="wt-text-body-01 wt-mr-xs-1"):
        if soup.find('p', class_="wt-text-body-01 wt-mr-xs-1").find_all('a', class_="wt-text-link-no-underline"):
            seller_href = soup.find('p', class_="wt-text-body-01 wt-mr-xs-1").find('a', class_="wt-text-link-no-underline")['href']
        else:
            seller_href = None
    else:
        seller_href = None
    
    if soup.find_all('h1', class_="wt-text-body-03 wt-line-height-tight wt-break-word wt-mb-xs-1"):
        listing_title = soup.find('h1', class_="wt-text-body-03 wt-line-height-tight wt-break-word wt-mb-xs-1").text.strip()
    else:
        listing_title = None
        
    if soup.find_all('p', class_="wt-text-title-03 wt-mr-xs-2"):
        listing_price = soup.find('p', class_="wt-text-title-03 wt-mr-xs-2").text
    else:
        listing_price = None

    listing_highlights = [
        _.text.strip() for _ 
        in soup.find_all('li', class_="wt-list-unstyled wt-display-flex-xs wt-align-items-flex-start")
    ]

    if soup\
        .find_all('div', class_="wt-display-inline-flex-xs wt-align-items-center wt-mb-xs-2 wt-flex-wrap"):
        if soup\
        .find('div', class_="wt-display-inline-flex-xs wt-align-items-center wt-mb-xs-2 wt-flex-wrap")\
        .find_all('span', class_='wt-text-caption'):
            store_sales = soup\
                .find('div', class_="wt-display-inline-flex-xs wt-align-items-center wt-mb-xs-2 wt-flex-wrap")\
                .find('span', class_='wt-text-caption').text
        else:
            store_sales = None
    else:
        store_sales = None
    
    if soup\
        .find_all('div', class_="wt-display-inline-flex-xs wt-align-items-center wt-mb-xs-2 wt-flex-wrap"):
        if soup\
        .find('div', class_="wt-display-inline-flex-xs wt-align-items-center wt-mb-xs-2 wt-flex-wrap")\
        .find_all('input', attrs={'name': 'initial-rating'}):
            store_rating = soup\
                .find('div', class_="wt-display-inline-flex-xs wt-align-items-center wt-mb-xs-2 wt-flex-wrap")\
                .find('input', attrs={'name': 'initial-rating'})['value']
        else:
            store_rating = None
    else:
        store_rating = None

    if soup.find_all('span', class_="wt-badge wt-badge--status-03 wt-mt-xs-1 wt-mr-xs-1 search-half-unit-mb"):
        listing_bestseller = soup\
            .find('span', class_="wt-badge wt-badge--status-03 wt-mt-xs-1 wt-mr-xs-1 search-half-unit-mb").text.strip()
    else:
        listing_bestseller = '-'

    # get description
    if soup.find_all('p', class_="wt-text-body-01 wt-break-word"):
        listing_desc = soup.find('p', class_="wt-text-body-01 wt-break-word").text
    else:
        listing_desc = None

    # GET REVIEWS FOR ITEM
    if soup.find_all('button', attrs={'aria-controls': 'same-listing-reviews-panel'}):
        listing_reviews = soup.find('button', attrs={'aria-controls': 'same-listing-reviews-panel'}).find('span').text.strip()
    else:
        listing_reviews = None
        
    # get shop reviews
    if soup.find_all('h3', class_="wt-mr-xs-2 wt-text-body-03"):
        store_reviews = soup.find('h3', class_="wt-mr-xs-2 wt-text-body-03").text.strip()
    else:
        store_reviews = None
        
    details = {
        'seller_text': seller_text,
        'seller_href': seller_href,
        'store_sales': store_sales,
        'store_rating': store_rating,
        'listing_title': listing_title,
        'listing_bestseller': listing_bestseller,
        'listing_price': listing_price,
        'listing_highlights': listing_highlights,
        'listing_desc': listing_desc,
        'listing_reviews': listing_reviews,
        'store_reviews': store_reviews,
        'cart': url_row[2],
        # 'url': url_row[1],
    }

    return details


# def get_search_listings(cat_url):

#     # get listing urls for cat or store url
#     r = requests.get(cat_url[0])

#     soup = BeautifulSoup(r.content)

#     search_listings = [
#         x.find('a')['href'] for x in 
#         soup.find(
#             'ul', 
#             class_="responsive-listing-grid wt-grid wt-grid--block wt-justify-content-flex-start wt-list-unstyled wt-pl-xs-0 tab-reorder-container"
#         )\
#         .find_all('li')
#     ]

#     next_page = soup.find('a', attrs={'data-page': 2})['href']
    
#     return search_listings, next_page


# def get_two_search(cat_url):
    
#     search_listings, next_page = get_search_listings(cat_url[0])
#     search_listing_results = search_listings

#     search_listings, next_page = get_search_listings(next_page)
#     search_listing_results.extend(search_listings)

#     return list(set(search_listing_results))


# def iterate_cat_urls(cat_urls):
    
#     data = []
#     for _ in tqdm(cat_urls):
#         results = get_two_search(_)
#         data.extend(results)

#     urls_clean = [
#         re.search(r'(?:(?!\?).)*', _).group(0) for _
#         in list(set(data))
#     ]
    
#     return urls_clean


def iterate_listings(urls_clean):
    
    data = []
    for _ in tqdm(urls_clean):
        
        try:
            url_result = get_listing_details(_)
            url_result.update({'url': _[1]})
            time.sleep(.2)
            data.append(url_result)
            json.dump(data, open('./data/temp_data.json', 'w')) # in case of interruption
            
        except KeyboardInterrupt:
            return data
    
    return data


def recover():
    data = json.load(open('temp_data.json', 'r'))
    return data

# def get_url_from_search(search_url, pages):

#     def request_urls(search_url):
        
#         # get clean URLs from search results
#         r = requests.get(search_url)

#         soup = BeautifulSoup(r.content)

#         clean_urls = [
#             _['href'] for _ in soup.find_all('a') if _.has_attr('href') and 'listing' in _['href']
#         ]
        
#         return clean_urls
    
#     clean_urls = request_urls(search_url)
    
#     for page in range(2, pages+1):
#         search_url = soup.select('a[href*="page={}"]'.format(page))[0]['href']
#         clean_urls.extend(request_urls(search_url))
#         time.sleep(1)
    
#     return clean_urls

In [4]:
def get_product_urls(shop_url):

    r = requests.get(shop_url)
    soup = BeautifulSoup(r.content)

    search_listings = [
        [
            _['title'], 
            re.search(r'(?:(?!\?).)*', _['href']).group(0), 
            _.find('div', class_="wt-text-brick wt-text-caption wt-pt-xs-1").text.strip()
        ]
        for _ in soup.find_all('a', class_="listing-link wt-display-inline-block wt-transparent-card")
        if _.find_all('div', class_="wt-text-brick wt-text-caption wt-pt-xs-1")
    ]

    # next_page = soup.find('a', attrs={'data-page': 2})['href']

    return search_listings

In [5]:
def iter_shops(shop_urls):
    
    shops = []
    for shop in tqdm(shop_urls):
        product_urls = get_product_urls(shop)
        shops.extend(product_urls)
    
    return shops

In [28]:
# For browsing shops
shop_urls = [
    'https://www.etsy.com/shop/ParadigmArt',
    'https://www.etsy.com/shop/DIYPaperBoutique'
    'https://www.etsy.com/shop/ParadigmArt'
    'https://www.etsy.com/shop/AngelicArtShop',
    'https://www.etsy.com/shop/OldMarketCorner',
    'https://www.etsy.com/shop/YouGrewPrintables',
    'https://www.etsy.com/shop/KAMDesignHaus',
    'https://www.etsy.com/shop/SurpriseINC',
    'https://www.etsy.com/shop/HelloMarketingStudio',
    'https://www.etsy.com/shop/SimpleBizBranding',
    'https://www.etsy.com/shop/ImperfectInspo',
    'https://www.etsy.com/shop/BrushAndBarley',
    'https://www.etsy.com/shop/GiftedGold',
    'https://www.etsy.com/shop/KIKIANDNIM',
    'https://www.etsy.com/shop/PrintsVilla',
    'https://www.etsy.com/shop/SevenPaperPrints',
    'https://www.etsy.com/shop/DaniellaDuranDesigns',
    'https://www.etsy.com/shop/ILovePrintable',
    'https://www.etsy.com/shop/DivineDigitalPrints',
    'https://www.etsy.com/shop/HEARTSinCOLORS',
    'https://www.etsy.com/shop/WallDecorIdeas',
]

In [29]:
product_urls = iter_shops(shop_urls)

In [30]:
product_urls[:5]

[['Work From Home Gifts, Do Epic Shit Sign, Work From Home Printable Wall Art, Print Art, Shirt, Motivation Wall Decor, Poster, WFH Gifts',
  'https://www.etsy.com/listing/486150960/work-from-home-gifts-do-epic-shit-sign',
  '12 people have this in their cart'],
 ['Inhale Exhale Print, Printable Wall Art Print, Instant Download Printable Art, Printable Quote, Prints, Motivation Wall Decor, Yoga Print',
  'https://www.etsy.com/listing/294341913/inhale-exhale-print-printable-wall-art',
  '20 people have this in their cart'],
 ['Wedding Photo Booth Prop Signs and Decorations - Rustic Wood Fence Wedding Photobooth Printables - Over 50 Images - Printable Digital File',
  'https://www.etsy.com/listing/522272710/wedding-photo-booth-prop-signs-and',
  'Over 20 people have this in their cart'],
 ['Wedding Photo Booth Prop Signs and Decorations - Black with White Writing Wedding Props - Over 50 Wedding Signs - Printable Digital File',
  'https://www.etsy.com/listing/613314189/wedding-photo-booth

In [31]:
# urls_clean = [_ for _ in product_urls]
data = iterate_listings(product_urls)

In [6]:
data = recover()

In [7]:
data[0]

{'seller_text': 'ParadigmArt',
 'seller_href': 'https://www.etsy.com/shop/ParadigmArt?ref=simple-shop-header-name&listing_id=486150960',
 'store_sales': '17,756 sales',
 'store_rating': '4.9483',
 'listing_title': 'Work From Home Gifts, Do Epic Shit Sign, Work From Home Printable Wall Art, Print Art, Shirt, Motivation Wall Decor, Poster, WFH Gifts',
 'listing_bestseller': '-',
 'listing_price': '\n                        $5.99\n                    ',
 'listing_highlights': ['Digital download', 'Digital file type(s): 5 JPG'],
 'listing_desc': '\n                    **STOREWIDE PROMOTION** Get 5 prints for the price of 3! Simply checkout with 5 prints in your cart and a 40% discount will automatically be applied. Happy shopping and printing!!INSTANT DOWNLOADNO PHYSICAL PRINT INCLUDEDPrint razor sharp prints at your own convenience in sizes up to A2!Have a custom quote in mind?  Choose from a style that you like from my selection of custom templates here → http://etsy.me/2uqfqRFDo check o

In [8]:
for _ in data:
    
    listing_price_clean = float(re.search(r'\$[0-9]{1,}\.[0-9]{1,}', _['listing_price']).group(0).replace('$', ''))
    store_sales_clean = int(re.search(r'[0-9,]{1,}', _['store_sales']).group(0).replace(',', ''))
    store_reviews_clean = int(re.search(r'[0-9,]{1,}', _['store_reviews']).group(0).replace(',', ''))
    listing_reviews_clean = int(_['listing_reviews']) if _['listing_reviews'] is not None else 0
    cart_clean = int(re.search(r'[0-9]{1,}', _['cart']).group(0))
    # listing_desc = _['listing_desc'].strip()
    
    _['listing_price_clean'] = listing_price_clean
    _['store_sales_clean'] = store_sales_clean
    _['store_reviews_clean'] = store_reviews_clean
    _['listing_reviews_clean'] = listing_reviews_clean
    _['cart_clean'] = cart_clean
    # _['listing_desc'] = listing_desc

df = pd.DataFrame(data)[[
    'seller_text', 'store_sales_clean', 'store_rating', 'store_reviews_clean', 'listing_reviews_clean',
    'listing_price_clean', 'listing_bestseller', 'listing_title', 
     'url', 'cart_clean', 'listing_desc', 'listing_highlights',
]]

In [9]:
df = df.rename(columns={
    'seller_text': 'seller',
    'store_sales_clean': 'sales (store)',
    'store_rating': 'rating (store)',
    'store_reviews_clean': 'reviews (store)',
    'listing_reviews_clean': 'reviews (listing)',
    'listing_price_clean': 'price',
    'listing_bestseller': 'bestseller',
    'listing_title': 'title',
    'listing_highlights': 'highlights',
    'cart_clean': 'cart',
    'listing_desc': 'desc'
})

In [10]:
df.sort_values(by=['cart', 'reviews (listing)', 'reviews (store)'], ascending=False).to_csv('./results/fire_listings_210709.csv', index=False)